### MLFlow and Optuna

* env used: generalml_p37_gpu_v1
* works also with CPU
* requires: pip install mlflow

In [46]:
import os

import optuna
import mlflow

# using a myconfig.py file I avoid to show pwd in NB
from myconfig import config

In [47]:
TRACK_SERVER_URI = "http://130.61.171.147:80"

# the key for succesfull auth is to set these two variables
# see documentation in https://www.mlflow.org/docs/latest/tracking.html
os.environ['MLFLOW_TRACKING_USERNAME'] = config["MLFLOW_TRACKING_USERNAME"]
os.environ['MLFLOW_TRACKING_PASSWORD'] = config['MLFLOW_TRACKING_PASSWORD']

In [48]:
mlflow.set_tracking_uri(TRACK_SERVER_URI)

exp_id = mlflow.set_experiment("exp2")

In [49]:
mlflow.start_run()

mlflow.log_param("a", 2)

In [50]:
mlflow.end_run()